In [24]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import yaml

import plotly.graph_objects as go


In [37]:
def bins(n):
    bins = []
    for i in range(n):
        bins.append(2**(i+1))
    return bins

def compute_cdf(a):
    x = np.sort(a);
    p = np.array(range(len(a)))/float(len(a)-1)
    return x, p

In [34]:
# Series of scores from my games.
jeff_scores = np.array([30080, 2460, 7396, 8280, 26420, 26304, 9836, 28012, 5260, 16540, 19148])

jeff_cdf_x, jeff_cdf_y = compute_cdf(jeff_scores)

In [47]:
with open('report.yml') as f:
    data = yaml.load(f.read())
    
# Cheat and read lookahead 5 from old report (it's slow to run)
with open('report_1.yml') as f:
    tmp = yaml.load(f.read())
    data['naive_lookahead5'] = tmp['naive_lookahead5']

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [48]:
def sortAlg(alg):
    # Weight by the average of the largest block
    return np.mean(np.array(bins(len(alg['largest_hist']))) * np.array(alg['largest_hist']))

sorted_names = sorted(data.keys(), key=lambda n: sortAlg(data[n]))

plt.figure()
for name in sorted_names:
    d = data[name]
    plt.plot(d['score_cdf_x'], d['score_cdf_y'], label=name)
plt.plot(jeff_cdf_x, jeff_cdf_y, label='Jeff')
plt.title('Final Score CDF')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:



bar_values = { k: d['largest_hist'] for k, d in data.items() }
max_size = max([len(x) for x in bar_values.values()])

# Pad any short arrays with zeros, and normalize
for k in bar_values.keys():
    missing = max_size - len(bar_values[k])
    if missing > 0:
        bar_values[k] += [0] * missing
    
    bar_values[k] = np.array(bar_values[k], dtype=np.float)
    bar_values[k] *= 100 / bar_values[k].sum()
    
print(bar_values)

labels = ["g"+str(x) for x in bins(max_size)]

print(labels)

fig = go.FigureWidget(data=[
    go.Bar(name=name, x=labels, y=bar_values[name]) 
    for name in sorted_names
    ])
# Change the bar mode
fig.update_layout(barmode='group')
#fig.show()
# for name, d in data.items():
#     plt.hist(d['largest_hist'], bins(len(d['largest_hist'])), label=name)
    
# plt.legend()

{'max_free_space': array([0.000e+00, 0.000e+00, 0.000e+00, 2.000e-02, 3.200e-01, 4.820e+00,
       3.416e+01, 4.698e+01, 1.338e+01, 3.200e-01, 0.000e+00]), 'max_free_space_3dir': array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.26,  5.64, 33.66, 45.98, 14.36,
        0.1 ,  0.  ]), 'naive_lookahead3': array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 4.000e-02,
       7.800e-01, 1.498e+01, 5.774e+01, 2.636e+01, 1.000e-01]), 'naive_lookaheadsorted3': array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.82, 11.56, 53.02,
       34.28,  0.32]), 'random': array([0.000e+00, 0.000e+00, 2.000e-02, 3.800e-01, 8.700e+00, 3.980e+01,
       4.436e+01, 6.720e+00, 2.000e-02, 0.000e+00, 0.000e+00]), 'random_3dir': array([ 0.06,  0.1 ,  0.34,  0.9 ,  9.52, 31.1 , 41.44, 15.76,  0.78,
        0.  ,  0.  ]), 'naive_lookahead5': array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 4.000e-02,
       4.000e-01, 6.500e+00, 4.474e+01, 4.690e+01, 1.420e+00])}
['g2', 'g4', 'g8', 'g16', 'g32', 'g64', 

FigureWidget({
    'data': [{'name': 'random',
              'type': 'bar',
              'uid': '50a98f8e-03c…